### <span style="color:red">NOTE: Needs Python 2.7</span> 

In [3]:
target_folder = "Training"
#target_folder = "Test"

# From (essay text) and To Files for the seq2seq model
from_file           = "/Users/simon.hughes/data/tensorflow/translate/cb/%s/inputs.txt" % target_folder
to_file             = "/Users/simon.hughes/data/tensorflow/translate/cb/%s/output.txt" % target_folder
to_file_most_freq   = "/Users/simon.hughes/data/tensorflow/translate/cb/%s/output_most_freq.txt"  % target_folder

In [4]:
from BrattEssay import load_bratt_essays
from collections import defaultdict
from IterableFP import flatten
from Settings import Settings

settings = Settings()
root_folder = settings.data_directory + "CoralBleaching/Thesis_Dataset/"
training_folder = root_folder + target_folder + "/"

essays = load_bratt_essays(training_folder)

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/
902 files found
902 essays processed


## 1. Flatten Essays, Separate Out Words From Tags

In [5]:
from collections import defaultdict

sentences, sent_tags = [], []
wd_freq, tag_freq = defaultdict(int), defaultdict(int)
for essay_ix, essay in enumerate(essays):
    for sent_ix, taggged_sentence in enumerate(essay.tagged_sentences):
        t_sentence = []
        sentences.append(t_sentence)
        t_tag_seq = []
        sent_tags.append(t_tag_seq)
        for word_ix, (wd, t) in enumerate(taggged_sentence):
            t_sentence.append(wd)
            t_tag_seq.append(t)
            wd_freq[wd]+=1
            for tag in t:
                tag_freq[tag] +=1

In [6]:
regular_tags = set((t for t in tag_freq.keys() if t[0].isdigit()))

In [7]:
print("Input  Vocabulary: " + str(len(wd_freq)))
print("Output Vocabulary: " + str(len(regular_tags)))
"""
Training:
Input  Vocabulary: 4286
Output Vocabulary: 13
"""

Input  Vocabulary: 4286
Output Vocabulary: 13


'\nTraining:\nInput  Vocabulary: 4286\nOutput Vocabulary: 13\n'

## 2. Collapse Tags to Most Frequent

In [67]:
regular_tag_seq = []
for tag_seq in sent_tags:
    r_tag_seq = []
    regular_tag_seq.append(r_tag_seq)
    # for each set of tags in sent
    for tag_set in tag_seq:
        tag_set = set((t for t in tag_set if t in regular_tags))
        r_tag_seq.append(tag_set)
del tag_set
del r_tag_seq

In [68]:
most_freq_tags = []
None_Tag = "None"
# for each sentence
for tag_seq in regular_tag_seq:
    most_freq = []
    most_freq_tags.append(most_freq)
    # for each set of tags in sent
    for tag_set in tag_seq:
        tag_set = set((t for t in tag_set if t in regular_tags))
        if len(tag_set) == 0:
            most_freq.append(None_Tag)
        else:
            tag = max(tag_set, key = lambda t:tag_freq[t])
            most_freq.append(tag)

### Validate All of the Lengths Match

In [69]:
len(sentences), len(most_freq_tags), len(regular_tag_seq)

(1918, 1918, 1918)

In [70]:
for ix, (sent, tags, mfreq_tags) in enumerate(zip(sentences, most_freq_tags, regular_tag_seq)):
    assert len(sent) == len(tags) == len(mfreq_tags), "Lengths differ at %i" % ix

## Save the Output For the Seq2SeqModel

In [71]:
def write_to_file(fname, sentences, mapper = lambda s:s):
    with open(fname, "w+") as f:
        for sent in sentences:
            mapped = map(mapper, sent)
            str_sent = " ".join(mapped).strip()
            f.write(str_sent + "\n")

In [72]:
write_to_file(from_file,           sentences=sentences)
write_to_file(to_file,             sentences=regular_tag_seq, mapper = lambda st: None_Tag if not st else ",".join(sorted(st)))
write_to_file(to_file_most_freq,   sentences=most_freq_tags)